<a href="https://colab.research.google.com/github/monsund/unsupervised-classification-of-Linkedin-Profiles-using-KMeans-LDA-TFIDF/blob/master/prod_version_random_forest_classification.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

In [ ]:
import pandas as pd
import pickle

In [ ]:
ranforest_model_filepath = '/content/drive/My Drive/NLP/with_swaraj/model/ranforest_model.pkl'
tfidf_model_filepath = '/content/drive/My Drive/NLP/with_swaraj/model/tfidf_model_profile_classification.pkl'

# **Import Model**

In [ ]:
with open(ranforest_model_filepath, "rb") as fp:   # Unpickling
  ranforest_model = pickle.load(fp)

with open(tfidf_model_filepath, "rb") as fp:   # Unpickling
  tfidf_model = pickle.load(fp)

In [ ]:
ranforest_model

RandomForestClassifier(bootstrap=True, ccp_alpha=0.0, class_weight=None,
                       criterion='gini', max_depth=None, max_features='auto',
                       max_leaf_nodes=None, max_samples=None,
                       min_impurity_decrease=0.0, min_impurity_split=None,
                       min_samples_leaf=1, min_samples_split=2,
                       min_weight_fraction_leaf=0.0, n_estimators=100,
                       n_jobs=None, oob_score=False, random_state=None,
                       verbose=0, warm_start=False)

In [ ]:
tfidf_model

TfidfVectorizer(analyzer='word', binary=False, decode_error='strict',
                dtype=<class 'numpy.float64'>, encoding='utf-8',
                input='content', lowercase=True, max_df=1.0, max_features=None,
                min_df=1, ngram_range=(1, 1), norm='l2', preprocessor=None,
                smooth_idf=True, stop_words=None, strip_accents=None,
                sublinear_tf=False, token_pattern='(?u)\\b\\w\\w+\\b',
                tokenizer=None, use_idf=True, vocabulary=None)

# **Remove Punctuation**

In [ ]:
#Function for removing punctuation and lowering case
def remove_punct(text):
  # define punctuation
  punctuations = '\'!()-[]{};:\"\,<>./?#$%^&*_~'
  
  for char in text:
    if char in punctuations:
      text = text.lower().replace(char,' ')    #lowering case
      text = text.replace('  ', ' ')
  return text

# **Remove Stopwords**

In [ ]:
import nltk
nltk.download('punkt')
from nltk.corpus import stopwords
nltk.download('stopwords')
from nltk.tokenize import word_tokenize

[nltk_data] Downloading package punkt to /root/nltk_data...
[nltk_data]   Unzipping tokenizers/punkt.zip.
[nltk_data] Downloading package stopwords to /root/nltk_data...
[nltk_data]   Unzipping corpora/stopwords.zip.


In [ ]:
# stop_words = set(stopwords.words('english'))
# additional_stop_words = set(['technology', 'development', 'experience', 'project', 'company'])

In [ ]:
# stop_words = stop_words.union(additional_stop_words)

In [ ]:
# Function removing stopwords-----------
def remove_stopwords(text):
  stop_words = set(stopwords.words('english'))
  additional_stop_words = set(['technology', 'development', 'experience', 'project', 'company'])
  stop_words = stop_words.union(additional_stop_words)

  word_tokens = word_tokenize(text)
  filtered_text = [w for w in word_tokens if not w in stop_words]
  data = (' '.join(filtered_text))
  return data

# **TDIDF Vectorization**

In [ ]:
from sklearn.feature_extraction.text import TfidfVectorizer


# **Function for finding Probability of Each Class**

In [ ]:
def get_class_prob(y_prob_list):
  topic_prob = []
  for i in range(len(y_prob_list)):
    topic3_prob = y_prob_list[i][0]
    topic4_prob = y_prob_list[i][1]
    topic5_prob = y_prob_list[i][2]
    topic7_prob = y_prob_list[i][3]
    topic8_prob = y_prob_list[i][4]
    topic10_prob = y_prob_list[i][5]
    topic11_prob = y_prob_list[i][6]
    topic12_prob = y_prob_list[i][7]
    topic14_prob = y_prob_list[i][8]
    
    dic = {'managerial field':topic3_prob, 'hr or people management':topic4_prob, 'marketing domain':topic5_prob, 'data science domain':topic7_prob,
          'product visionnaire, product management related field':topic8_prob, 'software domain':topic10_prob, 'related to investment,startups looking for investment':topic11_prob,
          'business,strategic growth,revenue':topic12_prob, 'entrepreneur or working with entrepreneur':topic14_prob}
    sort_dic = sorted(dic.items(), key=lambda x: x[1], reverse=True)
    topic_prob.append(sort_dic)
  return topic_prob  

# **Model Prediction**

In [ ]:
def get_profile_info(model, text):
  text_remove_punct = remove_punct(text)
  text_remove_stopwords = remove_stopwords(text_remove_punct)
  text_vec = tfidf_model.transform([text_remove_stopwords])
  class_prob = model.predict_proba(text_vec)

  profile_prob = get_class_prob(class_prob)
  return profile_prob[0]

In [ ]:
# topic_dic = {5: 'marketing domain', 8:'related to product visionnaire', 7:'related to data domain', 10:'software domain',14: 'entrepreneur or working with entrepreneur', 
#              11:'related to investment; startups looking for investment', 3:'managerial field', 12:'business,strategic growth,revenue', 4:'hr or people management'}

In [ ]:
text = '''HR Professional; having over 6 years of experience in managing a full spectrum of human resources programs, services, and functions with proven expertise in implementing the policies and procedures. '''
# tfidf_model.transform([text]).toarray()
get_profile_info(ranforest_model,text)

[('hr or people management', 0.43),
 ('software domain', 0.2),
 ('marketing domain', 0.18),
 ('managerial field', 0.09),
 ('data science domain', 0.05),
 ('product visionnaire, management related field', 0.03),
 ('related to investment,startups looking for investment', 0.01),
 ('business,strategic growth,revenue', 0.01),
 ('entrepreneur or working with entrepreneur', 0.0)]